In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression,ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import lightgbm 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', -1)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore")

In [2]:
train=pd.read_csv('train.csv')
test =pd.read_csv('test.csv')

In [3]:
train['Username'] = train['Username'].astype(object)
train.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [4]:
test['Username'] = test['Username'].astype(object)
test.head()

,ID,Tag,Reputation,Answers,Username,Views
0,366953,a,5645.0,3.0,50652,33200.0
1,71864,c,24511.0,6.0,37685,2730.0
2,141692,i,927.0,1.0,135293,21167.0
3,316833,i,21.0,6.0,166998,18528.0
4,440445,i,4475.0,10.0,53504,57240.0


In [5]:
test_ids = list(test.index)

In [6]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,330045.0,235748.682789,136039.418471,1.0,117909.0,235699.0,353620.0,471493.0
Reputation,330045.0,7773.146798,27061.412587,0.0,282.0,1236.0,5118.0,1042428.0
Answers,330045.0,3.917672,3.579515,0.0,2.0,3.0,5.0,76.0
Views,330045.0,29645.071342,80956.456478,9.0,2594.0,8954.0,26870.0,5231058.0
Upvotes,330045.0,337.505358,3592.441135,0.0,8.0,28.0,107.0,615278.0


In [7]:
test.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,141448.0,235743.073497,136269.867118,7.0,117797.0,235830.0,353616.00,471488.0
Reputation,141448.0,7920.927231,27910.724795,0.0,286.0,1245.0,5123.00,1042428.0
Answers,141448.0,3.914873,3.577460,0.0,2.0,3.0,5.00,73.0
Views,141448.0,29846.333112,80343.740166,9.0,2608.0,8977.0,26989.25,5004669.0


In [8]:
train.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [9]:
train.shape

(330045, 7)

In [10]:
test.shape

(141448, 6)

In [11]:
data = pd.concat([train,test])

temp = data.groupby(['Username']).agg({\
                                        "Answers":{\
#                                                   "sum_":'sum',
                                                  'mean_':'mean'
                                                },
                                        "Views":{\
#                                                   "sum_":'sum',
                                                  'mean_':'mean',
                                                }
                                       
                                       
                                       }).reset_index()
temp.columns =['Username','user_ans_mean','user_views_mean']
temp['UserAnsByView'] = 1000*temp['user_ans_mean']/temp['user_views_mean']
temp.head()

,Username,user_ans_mean,user_views_mean,UserAnsByView
0,0,6.0,37491.0,0.160038
1,1,20.0,1215345.0,0.016456
2,2,3.0,27153.0,0.110485
3,3,1.0,23464.0,0.042618
4,4,2.0,1061.0,1.885014


In [12]:
train = pd.merge(train,temp,on='Username')
train['AnsByUseransmean'] = train['Answers']/train['user_ans_mean']
train['ViewsByUserviewsmean'] = train['Views']/train['user_views_mean']
train['AnsByViews'] = 1000*train['Answers']/train['Views']
train.drop(['Username'],axis=1,inplace=True)
train = pd.get_dummies(train,columns=['Tag'])
train.index = train.ID
train.drop(['ID'],axis=1,inplace=True)
train.fillna(train.mean(),inplace=True)
train.head()

,Reputation,Answers,Views,Upvotes,user_ans_mean,user_views_mean,UserAnsByView,AnsByUseransmean,ViewsByUserviewsmean,AnsByViews,Tag_a,Tag_c,Tag_h,Tag_i,Tag_j,Tag_o,Tag_p,Tag_r,Tag_s,Tag_x
ID,,,,,,,,,,,,,,,,,,,,
52664,3942.0,2.0,7855.0,42.0,3.740741,31039.037037,0.120517,0.534653,0.253068,0.254615,1,0,0,0,0,0,0,0,0,0
320462,3942.0,7.0,26520.0,102.0,3.740741,31039.037037,0.120517,1.871287,0.854408,0.263952,0,0,0,0,1,0,0,0,0,0
352545,3942.0,2.0,2519.0,30.0,3.740741,31039.037037,0.120517,0.534653,0.081156,0.793966,0,0,0,0,0,0,0,0,1,0
128063,3942.0,2.0,8983.0,49.0,3.740741,31039.037037,0.120517,0.534653,0.289410,0.222643,0,0,0,0,0,0,1,0,0,0
58446,3942.0,5.0,44760.0,240.0,3.740741,31039.037037,0.120517,1.336634,1.442055,0.111707,0,0,0,1,0,0,0,0,0,0


In [13]:
test = pd.merge(test,temp,on='Username')
test['AnsByUseransmean'] = test['Answers']/test['user_ans_mean']
test['ViewsByUserviewsmean'] = test['Views']/test['user_views_mean']
test['AnsByViews'] = 1000*test['Answers']/test['Views']
test.drop(['Username'],axis=1,inplace=True)
test = pd.get_dummies(test,columns=['Tag'])
test.index = test.ID
test.drop(['ID'],axis=1,inplace=True)
test.fillna(test.mean(),inplace=True)
test.head()

,Reputation,Answers,Views,user_ans_mean,user_views_mean,UserAnsByView,AnsByUseransmean,ViewsByUserviewsmean,AnsByViews,Tag_a,Tag_c,Tag_h,Tag_i,Tag_j,Tag_o,Tag_p,Tag_r,Tag_s,Tag_x
ID,,,,,,,,,,,,,,,,,,,
366953,5645.0,3.0,33200.0,3.000000,22381.000000,0.134042,1.000000,1.483401,0.090361,1,0,0,0,0,0,0,0,0,0
71864,24511.0,6.0,2730.0,2.891892,9563.918919,0.302375,2.074766,0.285448,2.197802,0,1,0,0,0,0,0,0,0,0
297672,24511.0,2.0,3606.0,2.891892,9563.918919,0.302375,0.691589,0.377042,0.554631,0,1,0,0,0,0,0,0,0,0
369278,24511.0,1.0,12032.0,2.891892,9563.918919,0.302375,0.345794,1.258062,0.083112,0,1,0,0,0,0,0,0,0,0
404904,24511.0,3.0,2482.0,2.891892,9563.918919,0.302375,1.037383,0.259517,1.208703,0,1,0,0,0,0,0,0,0,0


In [14]:
#np.log1p(train["Reputation"]).plot(kind='hist')

In [15]:
# cols = ['Reputation','Views']
# for col in cols:
#     train[col] = np.log1p(train[col])

In [ ]:
X = train[[col for col in train.columns if col != 'Upvotes']]
y = train.Upvotes
kf = KFold(n_splits=10)


preds = np.zeros((len(test),1))

lr = LinearRegression()
rfc = RandomForestRegressor(n_estimators=10)
lgbm = lightgbm.LGBMRegressor(n_estimators=1000)
lasso =Lasso(alpha =0.0005, random_state=1)
ENet = ElasticNet(alpha=0.0005, l1_ratio=.9)
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

for clf in [lr,rfc,lgbm,lasso,ENet,KRR]:
    scores = []
    for train_idx,val_idx in kf.split(X,y):
        X_train,X_val = X.iloc[train_idx],X.iloc[val_idx]
        y_train,y_val = y.iloc[train_idx],y.iloc[val_idx]

        clf.fit(X_train,y_train)
        score = np.sqrt(mean_squared_error(y_val,clf.predict(X_val)))
        scores.append(score)

    #     pred = clf.predict(test).reshape(-1,1)
    #     preds += pred
    print(clf)
    print(round(np.mean(scores),2),end="\n")

# preds = preds/10
# print("\nmean score= ",np.mean(scores))

# temp = test.copy()
# temp["Upvotes"] = preds
# temp = temp[["Upvotes"]]
# temp.to_csv('try3.csv')

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
2657.94
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
1187.48
LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsa

In [391]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

NN_model = Sequential()

NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))

NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
NN_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 128)               2560      
_________________________________________________________________
dense_37 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_38 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_39 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 257       
Total params: 167,425
Trainable params: 167,425
Non-trainable params: 0
_________________________________________________________________


In [393]:
X = train[[col for col in train.columns if col != 'Upvotes']]
y = train.Upvotes
NN_model.fit(X, y, epochs=10, batch_size=32, validation_split = 0.2)

pred = NN_model.predict(test)
temp = test.copy()
temp["Upvotes"] = pred
temp = temp[["Upvotes"]]
temp.to_csv('try4.csv')

Train on 264036 samples, validate on 66009 samples
Epoch 1/10
264036/264036 [==============================] - 13s 49us/step - loss: 9869723.7501 - mean_squared_error: 9869723.0000 - val_loss: 367594.6981 - val_mean_squared_error: 367594.4375
Epoch 2/10
264036/264036 [==============================] - 13s 50us/step - loss: 9368908.7764 - mean_squared_error: 9368904.0000 - val_loss: 248922.2292 - val_mean_squared_error: 248922.3750
Epoch 3/10
264036/264036 [==============================] - 13s 48us/step - loss: 9895651.5177 - mean_squared_error: 9895656.0000 - val_loss: 17765145.1741 - val_mean_squared_error: 17765128.0000
Epoch 4/10
264036/264036 [==============================] - 13s 50us/step - loss: 9835929.2543 - mean_squared_error: 9835917.0000 - val_loss: 355834.5202 - val_mean_squared_error: 355834.7500
Epoch 5/10
264036/264036 [==============================] - 13s 48us/step - loss: 9703609.7047 - mean_squared_error: 9703594.0000 - val_loss: 380803.4016 - val_mean_squared_erro